In [15]:
############################## IMPORTS ##############################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Organisation des données

In [16]:
#convertir le csv en pandas dataframe
dataframe = pd.read_csv('export_IA.csv', sep=',', engine='c', encoding='utf-8', low_memory=False)

# Découverte des données

### Valeur cible (gravité de l’accident) pour la classification = classe

In [17]:
#compte les différents résultats pour chaque colonne
for i in range(len(dataframe.columns)):
    print("Valeur cible", i+1,  ": ", dataframe.columns[i], " - ", len(dataframe[dataframe.columns[i]].unique()))

Valeur cible 1 :  id_code_insee  -  8258
Valeur cible 2 :  Num_Acc  -  40250
Valeur cible 3 :  num_veh  -  58
Valeur cible 4 :  id_usa  -  73643
Valeur cible 5 :  date  -  29950
Valeur cible 6 :  ville  -  8078
Valeur cible 7 :  latitude.x  -  1253
Valeur cible 8 :  longitude.x  -  1495
Valeur cible 9 :  descr_cat_veh  -  24
Valeur cible 10 :  descr_agglo  -  2
Valeur cible 11 :  descr_athmo  -  9
Valeur cible 12 :  descr_lum  -  5
Valeur cible 13 :  descr_etat_surf  -  9
Valeur cible 14 :  description_intersection  -  9
Valeur cible 15 :  an_nais  -  101
Valeur cible 16 :  age  -  101
Valeur cible 17 :  place  -  10
Valeur cible 18 :  descr_dispo_secu  -  15
Valeur cible 19 :  descr_grav  -  4
Valeur cible 20 :  descr_motif_traj  -  6
Valeur cible 21 :  descr_type_col  -  7
Valeur cible 22 :  department_name  -  89
Valeur cible 23 :  department_number  -  89
Valeur cible 24 :  region_name  -  17


### Nombre d'instances

In [18]:
#compte le nombre d'instances et renvoie juste le nombre
print("Nombre d'instances : ", len(dataframe.index))

Nombre d'instances :  73643


### Nombre d'instances par classe

<Axes: xlabel='descr_grav'>

In [19]:

#creer un tableau qui affiche les données selon les descr_grav
graphique1 = dataframe.groupby('descr_grav').size()
#représenter en graphique 
graphique1.plot.bar()

### Taille des features

In [20]:
# Taille des features
print("Taille des features : ", len(dataframe.columns)-1)

Taille des features :  23


# Préparation des données

In [21]:
# On détecte l'OS et on execute la bonne commande pour la préparation des données de date
import platform
if platform.system() == 'Windows':
    # préparation des données, conversions des dates et des heures en entier sans les séparateurs
    dataframe['date'] = pd.to_datetime(dataframe['date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y%m%d%H%M%S').astype('Int64')
else :
    # préparation des données, conversions des dates et des heures en entier sans les séparateurs
    dataframe['date'] = pd.to_datetime(dataframe['date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y%m%d%H%M%S').astype(int)


print(dataframe.head())

  id_code_insee       Num_Acc num_veh  id_usa            date  \
0         10100  200900035904     A01  823507  20090803130000   
1         10100  200900004313     A01  752937  20090227223000   
2         10102  200900005197     A01  754945  20090113160000   
3         10102  200900005197     B02  754947  20090113160000   
4         10102  200900005206     A01  754970  20090209201500   

              ville  latitude.x  longitude.x  descr_cat_veh  descr_agglo  ...  \
0            CLEREY     48.2000          4.2              2            1  ...   
1            CLEREY     48.2000          4.2              3            2  ...   
2  COLOMBE LA FOSSE     48.2667          4.8              3            2  ...   
3  COLOMBE LA FOSSE     48.2667          4.8              3            2  ...   
4  COLOMBE LA FOSSE     48.2667          4.8              6            2  ...   

   an_nais   age  place                         descr_dispo_secu  descr_grav  \
0   1952.0  57.0    1.0  Utilisation d'une

In [22]:
# Transformartion des codes_insee en entier, en remplacant les A et B par 0
dataframe['id_code_insee'] = dataframe['id_code_insee'].str.replace('A', '0')
dataframe['id_code_insee'] = dataframe['id_code_insee'].str.replace('B', '0')

# On supprime la colonne 'num_veh' et 'ville' car elles ne nous sont pas utiles
dataframe = dataframe.drop('num_veh', axis=1)
dataframe = dataframe.drop('ville', axis=1)

In [23]:
# On tranforme la colonne 'descr_type_col' en entier
echelle = dataframe['descr_type_col'].unique()
dataframe['descr_type_col'] = dataframe['descr_type_col'].replace(echelle, [0,1,2,3,4,5,6])

In [24]:
# On transforme la colonne 'descr_dispo_secu' en entier
echelle = dataframe['descr_dispo_secu'].unique()
dataframe['descr_dispo_secu'] = dataframe['descr_dispo_secu'].replace(echelle, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

In [25]:
# On supprime la colonne 'department_name' car elle ne nous est pas utile
dataframe = dataframe.drop('department_name', axis=1)

In [26]:
# On ajoute une colonne 'region_number' qui contient le numéro de la région selon le region_name
REGIONS = {
    'auvergne-rhône-alpes': '84',
    'bourgogne-franche-comté': '27',
    'bretagne': '53',
    'centre-val de loire': '24',
    'corse': '94',
    'grand est': '44',
    'guadeloupe': '01',
    'guyane': '03',
    'hauts-de-france': '32',
    'île-de-france': '11',
    'la réunion': '04',
    'martinique': '02',
    'normandie': '28',
    'nouvelle-aquitaine': '75',
    'occitanie': '76',
    'pays de la loire': '52',
    'provence-alpes-côte d\'azur': '93'
}

# On ajoute une colonne 'region_number' qui contient le numéro de la région selon le region_name et REGIONS
# On met en minuscule les clés de régions pour éviter les erreurs
dataframe['region_number'] = dataframe['region_name'].replace(REGIONS)
dataframe = dataframe.drop('region_name', axis=1)

In [27]:
# On remplace le A et le B par un 0 dans le departement_number
dataframe['department_number'] = dataframe['department_number'].str.replace('A', '0')
dataframe['department_number'] = dataframe['department_number'].str.replace('B', '0')

In [28]:
dataframe.tail(100)

,id_code_insee,Num_Acc,id_usa,date,latitude.x,longitude.x,descr_cat_veh,descr_agglo,descr_athmo,descr_lum,...,description_intersection,an_nais,age,place,descr_dispo_secu,descr_grav,descr_motif_traj,descr_type_col,department_number,region_number
73543,97416,200900072581,905216,20090125165500,6.87387,43.6566,3,2,4,2,...,1,1967.0,42.0,1.204853,7,1,9,0,974,04
73544,97416,200900074573,909551,20091029081500,6.87387,43.6566,3,2,4,2,...,1,1947.0,62.0,1.204853,7,2,9,3,974,04
73545,97416,200900072724,905477,20090323191500,6.87387,43.6566,17,1,4,3,...,1,1992.0,17.0,1.000000,5,1,5,4,974,04
73546,97416,200900073649,907567,20090726171500,6.87387,43.6566,3,1,4,2,...,1,1986.0,23.0,1.000000,0,0,5,4,974,04
73547,97416,200900074565,909534,20091208080000,6.87387,43.6566,3,1,4,2,...,1,1961.0,48.0,1.000000,4,0,9,4,974,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73638,97423,200900073933,908050,20090810131500,55.29520,-21.1059,15,1,4,2,...,1,1979.0,30.0,1.000000,1,2,5,1,974,04
73639,97423,200900072904,905890,20090506063000,55.29520,-21.1059,2,1,4,1,...,1,1953.0,56.0,1.000000,0,0,1,1,974,04
73640,97424,200900073058,906209,20090621190000,55.46970,-21.1367,17,1,4,3,...,1,1962.0,47.0,1.000000,9,3,5,1,974,04
73641,97424,200900072573,905199,20090116221500,55.46970,-21.1367,15,2,4,3,...,1,1982.0,27.0,1.000000,6,2,5,3,974,04


In [ ]:
# On défini les valeurs de la colonne 'id_code_insee', 'department_number' et 'region_number' en entier

dataframe['id_code_insee'] = dataframe['id_code_insee'].astype(int)
dataframe['department_number'] = dataframe['department_number'].astype(int)
dataframe['region_number'] = dataframe['region_number'].astype(int)